In [1]:
import os, re
import pyconll
import json
import pandas as pd
import random
import openai
import os
import pandas as pd
import re
import json
from tqdm import tqdm

# import sys
# sys.path.append('../protosp01')


In [2]:
# navigate to protosp01 skill extraction directory to import libraries
if os.getcwd().split("/")[-1] == "notebooks":
    os.chdir("../protosp01/skillExtract")
assert os.getcwd().split("/")[-1] == "skillExtract"


from utils import *
from prompt_template import PROMPT_TEMPLATES

In [3]:
from split_words import Splitter

splitter = Splitter()
list(splitter.split_compound("Sprachkenntnisse")[0][1:])

['Sprach', 'Kenntnisse']

# get taxonomy elements

In [11]:
# navigate to data directory
if os.getcwd().split("/")[-1] == "skillExtract":
    os.chdir("../../data")
assert os.getcwd().split("/")[-1] == "data"

In [ ]:
# read from excel file each tab
# write to csv file each tab

taxonomy = pd.read_excel("taxonomy/KompetenzmodellKodierbuch.xlsx", sheet_name="V4")
print(len(taxonomy))
taxonomy = taxonomy.dropna(subset=["Type Level 1", "Type Level 2", "Definition"])
print(len(taxonomy))
taxonomy.insert(0, "unique_id", list(range(len(taxonomy))))
#TODO: make unique ID unique to the lowest level of the taxonomy

taxonomy.to_csv("taxonomy/taxonomy_V4.csv", index=False)

n = len(taxonomy)

tech = pd.read_excel(
    "taxonomy/KompetenzmodellKodierbuch.xlsx", sheet_name="Technologies"
)
print(len(tech))
certif = pd.read_excel(
    "taxonomy/KompetenzmodellKodierbuch.xlsx", sheet_name="Certifications"
)
print(len(certif))
lang = pd.read_excel("taxonomy/KompetenzmodellKodierbuch.xlsx", sheet_name="Languages")
print(len(lang))

tech_certif_lang = pd.concat([tech, certif, lang], ignore_index=True)
tech_certif_lang.insert(
    0, "unique_id", list([i + n for i in range(len(tech_certif_lang))])
)
# TODO: make unique ID increment from taxonomy ID

tech_certif_lang.head()
print(len(tech_certif_lang))
tech_certif_lang.to_csv("taxonomy/tech_certif_lang.csv", index=False)

In [6]:
tech_certif_lang.head()

,Level 1,Level 2,Level 1.5\n,unique_id
0,Technologies,Microsoft Excel,NaN,0
1,Technologies,Microsoft Office software,NaN,1
2,Technologies,Microsoft PowerPoint,NaN,2
3,Technologies,Microsoft Outlook,NaN,3
4,Technologies,Microsoft Word,NaN,4


# Extend taxonomy 

In [9]:
from api_key import *

generator_models = {
    "chatgpt": "gpt-3.5-turbo",
    "gpt-4": "gpt-4",
}

# API_KEY = ""
# ORGANIZATION = ""

openai.api_key = API_KEY

engine = "chatgpt"

In [10]:
PROMPT = "I am looking for occurrences of the <SKILL_TYPE> '<NAME>' in a document. However, the author doesn't always refer to this <SKILL_TYPE> using the full name. Generate only a list of 10 other names that I could look for, separated by commas."

EXAMPLE_DICT = {
    "technology": [
        "Microsoft Excel",
        "Excel, MS Excel, Microsoft Excel, Spreadsheet software by Microsoft, Microsoft's spreadsheet application, Excel program, Excel software, Microsoft's data analysis tool, Microsoft's workbook software, Spreadsheet program by Microsoft",
    ],
    "certification": [
        "AWS DevOps Engineer",
        "AWS, AWS DevOps Specialist, Amazon DevOps Engineer, AWS DevOps Practitioner, Certified AWS DevOps Professional, AWS DevOps Architect, Amazon Web Services DevOps Expert, AWS DevOps Solutions Engineer, AWS Cloud DevOps Engineer, AWS DevOps Deployment Specialist, AWS DevOps Integration Engineer",
    ],
}


class Generator:
    def __init__(self):
        self.model = generator_models[engine]

    def generate(self, skill_type, skill_name):
        messages = []
        sys_message = {
            "role": "system",
            "content": f"You are an expert at human resources, specialized in the IT domain.",
        }
        messages.append(sys_message)

        # Get the prompt

        question_example_content = PROMPT.replace("<SKILL_TYPE>", skill_type).replace(
            "<NAME>", EXAMPLE_DICT[skill_type][0]
        )
        example_answer = EXAMPLE_DICT[skill_type][1]
        question_content = PROMPT.replace("<SKILL_TYPE>", skill_type).replace(
            "<NAME>", skill_name
        )
        messages.append({"role": "user", "content": question_example_content})
        messages.append({"role": "assistant", "content": example_answer})
        messages.append({"role": "user", "content": question_content})
        flag = True
        while flag:  # while getting exception from API, retry
            try:
                response = openai.ChatCompletion.create(
                    model=self.model,
                    messages=messages,
                    top_p=1.0,
                    temperature=0.8,
                    frequency_penalty=0.8,
                    presence_penalty=0.5,
                )
                flag = False
            except Exception as e:
                print(f"The following error has happened. Waiting for 5seconds:\n{e}")
                time.sleep(5)
        output_text = response["choices"][0]["message"]["content"]
        return output_text

In [10]:
generator = Generator()

tech_certif_lang = pd.read_csv("taxonomy/tech_certif_lang.csv")
tech_certif_lang.drop(columns=["Level 1.5"], inplace=True)
tech = tech_certif_lang[tech_certif_lang['Level 1'] == "Technologies"]
tech['alternative_names'] = tech['Level 2'].apply(lambda name: generator.generate('technologie', name ))

certif = tech_certif_lang[tech_certif_lang["Level 1"] == "Certifications"]
certif["alternative_names"] = certif["Level 2"].apply(
    lambda name: generator.generate("certification", name)
)

/tmp/ipykernel_779043/232547023.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  certif['alternative_names'] = certif['Level 2'].apply(lambda name: generator.generate('certification', name ))


In [11]:
def clean_list(row):
    skill_name = row["Level 2"]
    skill_list = row["alternative_names"]
    skill_list = skill_list.replace("\n", ", ")
    skill_list = skill_list.split(":")[1] if ":" in skill_list else skill_list
    skill_list = re.sub(r"\d+\. ", "", skill_list)
    skill_list = skill_list.split(", ")
    skill_list = [
        skill for skill in skill_list if skill != "" and skill_name not in skill
    ]
    skill_list = ", ".join(skill_list)
    return skill_list

In [12]:
# tech['alternative_names_clean'] = tech.apply(clean_list, axis=1)
# to_save = tech[['Level 2', 'alternative_names_clean']]
# to_save.to_csv('taxonomy/technologies_alternative_names.csv', index=False, sep='\t')

certif["alternative_names_clean"] = certif.apply(clean_list, axis=1)
to_save = certif[["Level 2", "alternative_names_clean"]]
# to_save.to_csv('taxonomy/technologies_alternative_names.csv', index=False, sep='\t')

to_save.to_csv("taxonomy/certifications_alternative_names.csv", index=False, sep="\t")

/tmp/ipykernel_779043/1854031658.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  certif['alternative_names_clean'] = certif.apply(clean_list, axis=1)


In [37]:

pattern = r'\((.*?)\)'
def get_name(certif):
    matches = re.findall(pattern, certif)
    if matches:
        return matches[0]
    else:
        return ""

tech_certif_lang = pd.read_csv("../../data/taxonomy/tech_certif_lang.csv")
certif = tech_certif_lang[tech_certif_lang["Level 1"] == "Certifications"]

altnames = pd.read_csv("../../data/taxonomy/certifications_alternative_names.csv", sep="\t")
smaller_name = [get_name(name) for name in list(certif["Level 1.5"])]
smaller_name2 = [get_name(name) for name in list(certif["Level 2"])]

In [42]:
for index, row in altnames.iterrows():
    if smaller_name[index] != "":
        row['alternative_names_clean'] += ', ' + smaller_name[index]
    if smaller_name2[index] != "":
        row['alternative_names_clean'] += ', ' + smaller_name2[index]


In [43]:
altnames.to_csv("../../data/taxonomy/certifications_alternative_names.csv", index=False, sep="\t")

# merge taxonomy
**TODO: update directories for this section**

In [9]:
taxonomy = pd.read_csv("KompetenzmodellKodierbuch_V1.csv", sep="\t")
taxonomy.columns
taxonomy = taxonomy.dropna(subset=["Definition", "Type Level 3"])
taxonomy.head()

,ElementID,Dimension,Type Level 1,Type Level 2,Type Level 3,Type Level 4,Definition,Industry,Bildungsweg 1,Bildungsweg 2,Bildungsweg 3,Bildungsweg 4,Bildungsweg 5,Bildungsweg 6,Bildungsweg 7,Bildungsweg 8,Bildungsweg 9,Bildungsweg 10,Source,Connection SwissICT Model
1,2.1.1,Deskriptor,Fertigkeit,Kognitive Fertigkeit,Ideengenerierung,NaN,"Fertigkeit, Probleme zu erkennen und (viele un...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Gestaltung und Problemlösung
2,2.1.1.1,Deskriptor,Fertigkeit,Kognitive Fertigkeit,Ideengenerierung,Problembewusstsein,"Erkennen, wann etwas falsch ist oder wahrschei...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.onetonline.org/find/descriptor/res...,NaN
3,2.1.1.2,Deskriptor,Fertigkeit,Kognitive Fertigkeit,Ideengenerierung,Einfallsreichtum,Ideen zu einem Thema generieren (die Anzahl de...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.onetonline.org/find/descriptor/res...,NaN
4,2.1.1.3,Deskriptor,Fertigkeit,Kognitive Fertigkeit,Ideengenerierung,Originalität,"Ungewöhnliche, clevere, krative Ideen oder Lös...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.onetonline.org/find/descriptor/res...,NaN
5,2.1.1.4,Deskriptor,Fertigkeit,Kognitive Fertigkeit,Idea Generation and Reasoning,Deductive Reasoning,The ability to apply general rules to specific...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:
esco = pd.read_csv("skills_de.csv", sep=",")
print(esco.columns)
print(len(esco))
esco.head()

Index(['conceptType', 'conceptUri', 'skillType', 'reuseLevel',
       'preferredLabel', 'altLabels', 'hiddenLabels', 'status', 'modifiedDate',
       'scopeNote', 'definition', 'inScheme', 'description'],
      dtype='object')
13896


,conceptType,conceptUri,skillType,reuseLevel,preferredLabel,altLabels,hiddenLabels,status,modifiedDate,scopeNote,definition,inScheme,description
0,KnowledgeSkillCompetence,http://data.europa.eu/esco/skill/0005c151-5b5a...,skill/competence,sector-specific,Musikpersonal verwalten,NaN,NaN,released,2016-12-20T17:43:43Z,NaN,NaN,http://data.europa.eu/esco/concept-scheme/skil...,Zuweisen und Verwalten der Aufgaben des Person...
1,KnowledgeSkillCompetence,http://data.europa.eu/esco/skill/00064735-8fad...,skill/competence,occupation-specific,Strafvollzugsverfahren beaufsichtigen,NaN,NaN,released,2016-12-20T20:17:49Z,NaN,NaN,http://data.europa.eu/esco/concept-scheme/memb...,Überwachen des Betriebs einer Justizvollzugsan...
2,KnowledgeSkillCompetence,http://data.europa.eu/esco/skill/000709ed-2be5...,skill/competence,sector-specific,nicht unterdrückende Praktiken anwenden,NaN,NaN,released,2016-12-20T19:18:19Z,NaN,NaN,http://data.europa.eu/esco/concept-scheme/skil...,"Ermitteln von Repressionen in Gesellschaften, ..."
3,KnowledgeSkillCompetence,http://data.europa.eu/esco/skill/0007bdc2-dd15...,skill/competence,sector-specific,Einhaltung von Vorschriften von Eisenbahnfahrz...,NaN,NaN,released,2016-12-20T20:02:19Z,NaN,NaN,http://data.europa.eu/esco/concept-scheme/skil...,"Kontrollieren von Fahrzeugen, Komponenten und ..."
4,KnowledgeSkillCompetence,http://data.europa.eu/esco/skill/00090cc1-1f27...,skill/competence,cross-sector,verfügbare Dienste ermitteln,NaN,NaN,released,2016-12-20T20:15:17Z,NaN,NaN,http://data.europa.eu/esco/concept-scheme/memb...,Ermitteln der verschiedenen verfügbaren Dienst...


In [21]:
def concatenate_cols(row):
    output = row["preferredLabel"] if not pd.isna(row["preferredLabel"]) else ""
    output += f" {row['altLabels']}" if not pd.isna(row["altLabels"]) else ""
    output += f" {row['hiddenLabels']}" if not pd.isna(row["hiddenLabels"]) else ""
    return output


esco = esco[esco["skillType"] == "knowledge"]
print(len(esco))
esco["ElementID"] = [1] * len(esco)
esco["Type Level 3"] = esco.apply(concatenate_cols, axis=1)

esco.rename(
    columns={
        "conceptUri": "Source",
        "skillType": "Dimension",
        "description": "Definition",
    },
    inplace=True,
)

esco = esco[["ElementID", "Source", "Dimension", "Definition", "Type Level 3"]]

esco.head()

3059


,ElementID,Source,Dimension,Definition,Type Level 3
7,1,http://data.europa.eu/esco/skill/000f1d3d-220f...,knowledge,Techniken und Grundsätze der Softwareentwicklu...,Haskell
16,1,http://data.europa.eu/esco/skill/00506f28-e884...,knowledge,Vorbeugung und Behandlung von Verletzungen ode...,Sport- und Bewegungsmedizin
24,1,http://data.europa.eu/esco/skill/00920055-eb50...,knowledge,Eine auf einem Plan basierende Unternehmensstr...,Unternehmensrisikomanagement
30,1,http://data.europa.eu/esco/skill/00c04e40-35ea...,knowledge,Das Modell der inkrementellen Entwicklung ist ...,inkrementelle Entwicklung
31,1,http://data.europa.eu/esco/skill/00c51318-4ea9...,knowledge,"Arten von Spezialausrüstung, Prothesen und Ort...",Hilfsmittel für Alltagsarbeiten


In [24]:
# append dataframe to taxonomy
taxonomy2 = taxonomy.append(esco, ignore_index=True)
taxonomy2.tail()
len(taxonomy2)

/tmp/ipykernel_82526/3048395446.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  taxonomy2 = taxonomy.append(esco, ignore_index=True)


3458

In [49]:
taxonomy2.to_csv("taxonomy_with_esco.csv", index=False, sep="\t")

# load jobs and course data

- https://evrlearn-staging.herokuapp.com/api/v1/proto/learning_opportunities - list of courses
- https://evrlearn-staging.herokuapp.com/api/v1/proto/vacancies - list of parsed jobs
- https://evrlearn-staging.herokuapp.com/api/v1/proto/job_skills - list of available job skills
- https://evrlearn-staging.herokuapp.com/api/v1/proto/job_profiles - list of job profiles with related skills 


### Run from the protosp01 directory:

```bash
bash update_platform_data.sh
```

In [21]:
# !wget https://evrlearn-staging.herokuapp.com/api/v1/proto/learning_opportunities -O "/home/montario/Documents/EPFL - NLP/SCESC/code/raw/learning_opportunities"
# !wget https://evrlearn-staging.herokuapp.com/api/v1/proto/vacancies -O "/home/montario/Documents/EPFL - NLP/SCESC/code/raw/vacancies"

In [22]:
# load json file
job_profiles = pd.read_json("raw/job_profiles.json")
job_profiles.head()
len(job_profiles)

460

In [23]:
job_skills = pd.read_json("raw/job_skills.json")
len(job_skills)  # 35

398

In [24]:
courses = pd.read_json("raw/learning_opportunities.json")
print(len(courses))  # 5738
# TODO take only active = True courses
print(len(courses[courses["active"] == True]))  # 4549
courses.columns

5738
4549


Index(['id', 'name', 'type', 'intro', 'active', 'pricing_description',
       'ects_points', 'average_effort_per_week', 'total_effort',
       'key_benefits', 'learning_targets_description',
       'target_group_description', 'structure_description',
       'application_process_description', 'admission_criteria_description',
       'required_number_years_of_experience', 'level', 'certificate_type',
       'currency', 'pricing_type', 'transaction_type'],
      dtype='object')

In [25]:
jobs = pd.read_json("raw/vacancies.json", encoding="utf-8")
print(len(jobs))  # 3759 --> 6616
jobs.head()

7631


,id,name,orig_url,company_name,company_url,lead,description
0,4623,Business Analyst Leistungen,https://www.jobs.ch/en/vacancies/detail/70cabf...,ELCA Informatik AG,https://www.jobs.ch/en/companies/11529-elca-in...,None,Um unser Wachstum weiterhin erfolgreich fortzu...
1,15,Securities Master Data Specialist (Ref. 2177),https://www.jobs.ch/en/vacancies/detail/0f8346...,None,None,None,"<!DOCTYPE html>\n<html lang=""en"" data-template..."
2,2226,IT Supporter (m/w),https://www.jobs.ch/en/vacancies/detail/e72b68...,univativ Schweiz AG,https://www.jobs.ch/en/companies/95851-univati...,"Ob Student, Berufseinsteiger oder Berufserfahr...",Karrierelevel: Berufseinsteiger Pensum: 80-10...
3,5,Scrum Master/-in & Applikations-Analyst/-in Ep...,https://www.jobs.ch/en/vacancies/detail/9a1fa7...,None,None,None,Das Luzerner Kantonsspital mit den Standorten ...
4,16,"Product Owner ""Financial Messaging"" for Multib...",https://www.jobs.ch/en/vacancies/detail/703709...,None,None,None,"<!DOCTYPE html>\n<html lang=""en"" data-template..."


In [26]:
jobs["fulltext"] = jobs["description"] + jobs["name"]
courses["fulltext"] = (
    courses["structure_description"].fillna("")
    + courses["name"]
    + courses["key_benefits"].fillna("")
    + courses["intro"].fillna("")
)


def count_sentences(text):
    sentences = re.split(r"(?<=[.!?]) +", text)
    return len(sentences)


price_per_sentence_gpt3 = 0.035 / 32
price_per_sentence_gpt4 = 1.2 / 32

# Apply the function to create a new column 'num_sentences'
for dfname, df in zip(["jobs", "courses"], [jobs, courses]):
    df["num_sentences"] = df["fulltext"].apply(count_sentences)

    # Calculate the average number of sentences
    average_sentences = df["num_sentences"].mean()
    total_sentences = df["num_sentences"].sum()

    price_with_gpt3 = total_sentences * price_per_sentence_gpt3
    prince_with_gpt4 = total_sentences * price_per_sentence_gpt4

    print(dfname, ": \n")
    print(f"Average number of sentences: {average_sentences:.2f}")
    print(f"Total number of sentences: {total_sentences:.2f}")
    print(f"price_with_gpt3: {price_with_gpt3:.2f}")
    print(f"prince_with_gpt4: {prince_with_gpt4:.2f}")
    print("\n")

jobs : 

Average number of sentences: 12.31
Total number of sentences: 93935.00
price_with_gpt3: 102.74
prince_with_gpt4: 3522.56


courses : 

Average number of sentences: 6.35
Total number of sentences: 36412.00
price_with_gpt3: 39.83
prince_with_gpt4: 1365.45




In [27]:
cols = ["structure_description", "name", "key_benefits", "intro"]
for col in cols:
    print(col, courses[col].isna().sum())

cols = ["description", "name"]
for col in cols:
    print(col, jobs[col].isna().sum())

structure_description 1621
name 0
key_benefits 17
intro 27
description 0
name 0


In [28]:
courses.describe()

,id,ects_points,average_effort_per_week,total_effort,required_number_years_of_experience,num_sentences
count,5738.000000,1412.000000,1179.000000,5240.000000,585.000000,5738.000000
mean,5220.989718,25.844901,12.579304,154.477672,2.842735,6.345765
std,2643.716068,237.474303,22.024206,355.025285,1.698877,7.225278
min,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000
25%,3788.250000,10.000000,5.000000,3.000000,2.000000,3.000000
50%,5781.500000,15.000000,10.000000,16.000000,2.000000,5.000000
75%,7471.750000,15.000000,16.000000,130.000000,3.000000,8.000000
max,8906.000000,8900.000000,450.000000,5400.000000,15.000000,205.000000


In [30]:
# sample 20 sentences:
jobs_sampled = jobs.sample(30)["description"].tolist()
jobs_sampled = [text for text in jobs_sampled if detect_language(text) == "de"]
jobs_sampled = [re.split(r"(?<=[.!?]) +", job) for job in jobs_sampled]
jobs_sampled = [
    random.sample([sent for sent in sent_list if 20 < len(sent) < 10000], 1)[0]
    for sent_list in jobs_sampled
]
print(jobs_sampled)
# jobs_json = [{'sentence': sent, 'extracted_skills': None} for sent in jobs_sampled]
# save as json
# with open("raw/job_sentences.json", "w", encoding="utf-8") as f:
#     json.dump(jobs_json, f, ensure_ascii=False, indent=4)

['Wir begleiten unsere Kunden von der ersten Idee bis hin zur Serienfertigung inmittleren bis grossen Stückzahlen und suchen per sofort Verstärkung für unser Team als: Leiter*in Qualitätsmanagement / Entwicklungsingenieur*in  Deine Hauptaufgaben:   \n Durchführen der QM-Maßnahmen gemäss DIN ISO 9001:2015 \n Koordinieren von Aktivitäten zur Überwachung und Verbesserung des QM-Systems sowie Kontrolleder Umsetzung der beschlossenen Massnahmen \n Bearbeiten von Reklamationen und Fehlern (intern und extern).', 'Mehr als 650 Mitarbeitende an unserem Hauptsitz in Zürich sowie in den Büros in Bern, Lausanne, Budapest, Lissabon, Singapur und Ho-Chi-Minh-Stadt entwickeln Lösungen, die unser Versprechen «And your digital business works» an unsere Kunden erfüllen.', 'Auch im Vertrieb von ganzheitlichen Lösungen und im IT-Projektgeschäft sind wir ein bekannter Ansprechpartner.', 'Klicke dafür auf «Jetzt bewerben».', 'Corporate Social Responsibility sowie DEI-Standards (Diversity, Ethics and Inclusi

# extract skillspan data

**TODO: update directories for this section**

In [45]:
path = "SkillSpan/data/Skills/skillspan_tech_test.conll"

# read conll file
with open(path, "r") as f:
    data = f.read()


def clean(sentence, output_skills=False):
    tokens = [tok.split("\t")[0] for tok in sentence.split("\n")]
    tokens_clean = " ".join(tokens)
    token_types = [
        tok.split("\t")[2] if len(tok.split("\t")) > 1 else "O"
        for tok in sentence.split("\n")
    ]
    skills = []
    current_skill = ""
    for token, type in zip(tokens, token_types):
        if type[0] == "B":
            current_skill += token
        elif type[0] == "I":
            current_skill += " " + token
        elif type[0] == "O":
            if current_skill != "":
                skills.append(current_skill)
                current_skill = ""
    if output_skills:
        if skills:
            print(skills)
        return (tokens_clean, skills)
    else:
        return tokens_clean


# data = data.replace('\n\n\n', '###\n')
paragraphs = data.split("###\n")
sentences = [p.split("\n\n") for p in paragraphs]
sentences_clean = [[clean(s) for s in p if s != ""] for p in sentences]
sentence_skills = [
    [clean(s, output_skills=True) for s in p if s != ""] for p in sentences
]

# print to txt file:
with open("skillspan_tech_test.txt", "w") as f:
    for p in sentences_clean:
        for s in p:
            f.write(s + "\n")
        f.write("\n*********************************************************\n")

# save to csv file:
for i, joboffer in enumerate(sentence_skills):
    for sentence, skill in joboffer:
        print(i, sentence, skill)
        with open("skillspan_tech_test.csv", "a") as f:
            f.write(str(i) + "\t" + sentence + "\t" + str(skill) + "\n")

[' innovation and engineering practices']
[' software development lifecycle']
[' BS/BA degree']
[' architecture', ' design']
[' Java', ' Javascript']
[' business analysis development maintenance']
[' HTML5']
[' Node JS ( server side and tooling ) *', ' React']
[' innovation and engineering practices']
[' software development lifecycle']
[' BS/BA degree']
[' Java Spring Boot NoSQL Message Cloud CI/CD']
[' architecture', ' design']
[' business analysis']
[' automation', ' Azure PaaS microservices . NET PowerShell', ' MS SQL']
[' CI', ' Azure PaaS']
[' Azure', ' subscriptions resource groups', ' DevOps', ' GitHub']
[' SonarQube . NET Analyzers Infer#']
[' Azure AppService IIS )']
[' Azure AppService', ' . NET']
[' automation']
[' based templates']
[' Azure SQL']
[' Azure DevOps ( cloud and server )']
[' Docker Containers', ' K8s']
[' English']
[' innovation & engineering practices']
[' software development lifecycle']
[' BS/BA degree']
[' application data and infrastructure architecture']

In [ ]:
sentences = []
for sentence in train:
    for token in sentence:
        if token.upos == "AUX" and (
            token.head != "0" and sentence[token.head].upos == "NOUN"
        ):
            review_sentences.append(sentence)

print("Review the following sentences:")
for sent in review_sentences:
    print(sent.id)